In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from matplotlib.patches import Ellipse

# Function to draw an ellipse for a 2D Gaussian
def draw_ellipse(position, covariance, ax, n_std=3, **kwargs):
    # Calculate the eigenvectors and eigenvalues of the covariance matrix
    eigenvals, eigenvecs = np.linalg.eigh(covariance)
    # Compute the angle between the x-axis and the largest eigenvector
    angle = np.degrees(np.arctan2(*eigenvecs[:, 0][::-1]))
    # Compute the width and height of the ellipse
    width, height = 2 * n_std * np.sqrt(eigenvals)
    # Create the ellipse and add it to the plot, passing angle as a keyword argument
    ellipse = Ellipse(position, width=width, height=height, angle=angle, **kwargs)
    ax.add_patch(ellipse)

# Data Generation
np.random.seed(0)
n_samples = 300
true_mean1 = [0, 0]
true_mean2 = [5, 5]
true_cov1 = [[1, 0.7], [0.7, 1]]
true_cov2 = [[1.5, -0.9], [-0.9, 1.5]]
data = np.concatenate((np.random.multivariate_normal(true_mean1, true_cov1, int(0.4 * n_samples)),
                       np.random.multivariate_normal(true_mean2, true_cov2, int(0.6 * n_samples))))

# EM Algorithm Implementation
mean1, mean2 = np.array([-30, -50]), np.array([20, 10])
cov1, cov2 = np.array([[10, 0], [0, 10]]), np.array([[10, 0], [0, 20]])
log_likelihood = []

max_iter = 30
for iteration in range(max_iter):
    # E-step
    resp1 = multivariate_normal.pdf(data, mean1, cov1)
    resp2 = multivariate_normal.pdf(data, mean2, cov2)
    gamma = resp1 / (resp1 + resp2)
    
    # M-step
    mean1 = np.dot(gamma, data) / np.sum(gamma)
    cov1 = (np.dot((gamma[:, np.newaxis] * (data - mean1)).T, (data - mean1)) 
            / np.sum(gamma))
    
    mean2 = np.dot((1 - gamma), data) / np.sum(1 - gamma)
    cov2 = (np.dot(((1 - gamma)[:, np.newaxis] * (data - mean2)).T, (data - mean2)) 
            / np.sum(1 - gamma))
    
    # Compute log-likelihood
    ll_current = np.sum(np.log(resp1 * gamma + resp2 * (1 - gamma)))
    log_likelihood.append(ll_current)

    # Visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), gridspec_kw={'height_ratios': [3, 1]}, dpi=200)
    
    # Plot data and Gaussians
    ax1.scatter(data[:, 0], data[:, 1], c=gamma, cmap='coolwarm', marker='o')
    ax1.scatter(mean1[0], mean1[1], c='red', marker='s', s=50, linewidths=0.5, edgecolors='white', label='Mean 1')
    ax1.scatter(mean2[0], mean2[1], c='blue', marker='s', s=50, linewidths=0.5, edgecolors='white', label='Mean 2')
    
    # Draw 3-sigma ellipses
    draw_ellipse(mean1, cov1, ax1, n_std=3, edgecolor='red', facecolor='none', linestyle='dashed', label='3σ Bound Mean 1')
    draw_ellipse(mean2, cov2, ax1, n_std=3, edgecolor='blue', facecolor='none', linestyle='dashed', label='3σ Bound Mean 2')
    
    ax1.set_title(f'Iteration {iteration + 1}, Log-Likelihood: {ll_current:.3f}')
    ax1.legend()

    # Plot log-likelihood
    ax2.plot(log_likelihood, '-o', label='Log-Likelihood', markersize=5)
    ax2.legend()
    
    # Determine the data range for proper visualization
    data_min = np.min(data, axis=0)
    data_max = np.max(data, axis=0)
    
    width1 = np.sqrt(max(np.linalg.eigh(cov1)[0]))
    width2 = np.sqrt(max(np.linalg.eigh(cov2)[0]))
    
    # Adjust the plot limits
    x_min = min(data_min[0] - width1, data_min[0] - width2)
    x_max = max(data_max[0] + width1, data_max[0] + width2)
    y_min = min(data_min[1] - width1, data_min[1] - width2)
    y_max = max(data_max[1] + width1, data_max[1] + width2)
    
    ax1.set_xlim(x_min, x_max)
    ax1.set_ylim(y_min, y_max)

    plt.show()
